In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [1]:
#! git clone https://github.com/Tikquuss/meta_XLM /content/meta_XLM

In [ ]:
#! cp -R /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs /content

In [2]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [3]:
%cd tools
! git clone https://github.com/moses-smt/mosesdecoder
! git clone https://github.com/glample/fastBPE && cd fastBPE && g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

/home/jupyter/meta_XLM/XLM/tools
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 147544 (delta 12), reused 15 (delta 5), pack-reused 147514
Receiving objects: 100% (147544/147544), 129.75 MiB | 30.91 MiB/s, done.
Resolving deltas: 100% (113998/113998), done.
Cloning into 'fastBPE'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 59 (delta 0), reused 2 (delta 0), pack-reused 54
Unpacking objects: 100% (59/59), done.


In [4]:
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [6]:
%env OUTPATH=/home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [8]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [9]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster3/data/Ghomala_Limbum/processed/Ghomala-Limbum.Ghomala.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6333


In [10]:
%env epoch_size=6333

%env lgs=Ghomala-Limbum

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6333
env: lgs=Ghomala-Limbum
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [ ]:
%env mlm_steps=Ghomala,Limbum,Ghomala-Limbum
%env batch_size=8
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster3
! python train.py --exp_name mlm_tlm_GhomalaLimbum --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Ghomala,Limbum,Ghomala-Limbum
env: batch_size=8
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster3
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/20/20 08:46:41 - 0:00:00 - ============ Initialized logger ============
INFO - 05/20/20 08:46:41 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 8
                                     beam_size: 1
                     

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_GhomalaLimbum  /content/drive/"My Drive"/African_Translator/models/africa/cluster3

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [ ]:
%env epoch_size=6360
%env lgs=Ghomala-Limbum

env: epoch_size=6360
env: lgs=Bafia-Bulu


In [ ]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [ ]:
%env stopping_criterion=valid_mt_bleu,10
%env validation_metrics=valid_mt_bleu
%env reload_model=dumped/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_GhomalaLimbum/maml/best-valid_mlm_ppl.pth
%env ae_steps=Ghomala,Limbum
%env bt_steps=Ghomala-Limbum-Ghomala,Limbum-Ghomala-Limbum

env: stopping_criterion=valid_mt_bleu,10
env: validation_metrics=valid_mt_bleu
env: reload_model=dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
env: ae_steps=Bafia,Bulu
env: bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu


In [ ]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# unsupervised MT
! python train.py --exp_name UnSupMT_GhomalaLimbum --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [ ]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# supervised MT
%env mt_steps=Ghomala-Limbum,Limbum-Ghomala          
! python train.py --exp_name SupMT_GhomalaLimbum --exp_id $exp_id  --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/SupMT_GhomalaLimbum  /content/drive/"My Drive"/African_Translator/models/africa/cluster3